# Loading Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Libraries

In [2]:
%matplotlib inline
import pandas as pd
import os, cv2
import numpy as np
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from skimage.measure import regionprops
from sklearn import preprocessing

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from tqdm import tqdm
from skimage.io import imread
from matplotlib.pyplot import imread

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

# Loading Dataset

In [3]:
def load_data_original_signatures(data_dir):
    for img in tqdm(os.listdir(data_dir)):
        if(img.endswith(".png")):
          path = os.path.join(data_dir,img)
          labels_original.append(str(img))
          img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
          data_original.append(img)

def load_data_forged_signatures(data_dir):
    for img in tqdm(os.listdir(data_dir)):
      if(img.endswith(".png")):
        path = os.path.join(data_dir,img)
        labels_forged.append(str(img))
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        data_forged.append(img)

In [4]:
original_signatures = '/content/drive/MyDrive/BTP/CEDAR/full_org'
forged_signatures = '/content/drive/MyDrive/BTP/CEDAR/full_forg'

In [5]:
data_original = []
data_forged = []
labels_original = []
labels_forged = []

load_data_original_signatures(original_signatures)
load_data_forged_signatures(forged_signatures)

100%|██████████| 1321/1321 [00:10<00:00, 126.68it/s]


In [6]:
data_original = np.array(data_original)
data_forged = np.array(data_forged)

<ipython-input-6-fa221fb6123a>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_original = np.array(data_original)
<ipython-input-6-fa221fb6123a>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_forged = np.array(data_forged)


# Preprocessing

## Binarizing Image

In [7]:
def binarization(img):
  (thresh, im_bw) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  return np.invert(im_bw)

## Cropping Image

In [8]:
def crop_image(img,tol=0):
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

## Resizing Image

In [9]:
def resize_image(img):
  img = cv2.resize(img, (224, 224))
  return img

## Combining all preprocessing techniques

In [10]:
for i in range(len(data_original)):
  img = data_original[i]
  binarized_img = binarization(img)
  cropped_img = crop_image(binarized_img)
  resized_img = resize_image(cropped_img)
  data_original[i] = resized_img

In [11]:
for i in range(len(data_forged)):
  img = data_forged[i]
  binarized_img = binarization(img)
  cropped_img = crop_image(binarized_img)
  resized_img = resize_image(cropped_img)
  data_forged[i] = resized_img

# Feature Extraction

In [12]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224,3))

for layer in base_model.layers:
	layer.trainable = False

In [13]:
training_data = pd.DataFrame(columns = ["signature", "signature_class"])

for i in range(1,45):
  person_original_mask = np.char.startswith(labels_original, "original_" + str(i) + "_")
  person_forged_mask = np.char.startswith(labels_forged, "forgeries_" + str(i) + "_")
  person_original_signatures = data_original[person_original_mask]
  person_forged_signatures = data_forged[person_forged_mask]

  for j in range(24):
    stacked_img = np.dstack((person_original_signatures[j], person_original_signatures[j], person_original_signatures[j]))
    input_img = np.expand_dims(stacked_img, axis = 0)
    feature_matrix = base_model.predict(input_img)
    flattened_matrix = feature_matrix.reshape(-1)
    training_data.loc[len(training_data)] = [flattened_matrix, 1]

  for j in range(24):
    stacked_img = np.dstack((person_forged_signatures[j], person_forged_signatures[j], person_forged_signatures[j]))
    input_img = np.expand_dims(stacked_img, axis = 0)
    feature_matrix = base_model.predict(input_img)
    flattened_matrix = feature_matrix.reshape(-1)
    training_data.loc[len(training_data)] = [flattened_matrix, 0]

training_data.head()

1/1 [==============================] - 0s 488ms/step


,signature,signature_class
0,"[0.0, 0.0, 0.0, 0.0, 7.877913, 0.0, 0.0, 0.0, ...",1
1,"[0.0, 0.0, 0.0, 0.0, 6.8483105, 0.0, 0.0, 0.0,...",1
2,"[0.0, 0.0, 0.0, 0.0, 3.8534906, 0.0, 0.0, 0.0,...",1
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
4,"[0.0, 0.0, 0.0, 0.0, 28.147108, 0.0, 0.0, 0.0,...",1


In [14]:
testing_data = pd.DataFrame(columns = ["signature", "signature_class"])

for i in range(44,56):
  person_original_mask = np.char.startswith(labels_original, "original_" + str(i) + "_")
  person_forged_mask = np.char.startswith(labels_forged, "forgeries_" + str(i) + "_")
  person_original_signatures = data_original[person_original_mask]
  person_forged_signatures = data_forged[person_forged_mask]

  for j in range(24):
    stacked_img = np.dstack((person_original_signatures[j], person_original_signatures[j], person_original_signatures[j]))
    input_img = np.expand_dims(stacked_img, axis = 0)
    feature_matrix = base_model.predict(input_img)
    flattened_matrix = feature_matrix.reshape(-1)
    testing_data.loc[len(testing_data)] = [flattened_matrix, 1]

  for j in range(24):
    stacked_img = np.dstack((person_forged_signatures[j], person_forged_signatures[j], person_forged_signatures[j]))
    input_img = np.expand_dims(stacked_img, axis = 0)
    feature_matrix = base_model.predict(input_img)
    flattened_matrix = feature_matrix.reshape(-1)
    testing_data.loc[len(testing_data)] = [flattened_matrix, 0]

1/1 [==============================] - 0s 481ms/step


In [15]:
training_data.head()

,signature,signature_class
0,"[0.0, 0.0, 0.0, 0.0, 7.877913, 0.0, 0.0, 0.0, ...",1
1,"[0.0, 0.0, 0.0, 0.0, 6.8483105, 0.0, 0.0, 0.0,...",1
2,"[0.0, 0.0, 0.0, 0.0, 3.8534906, 0.0, 0.0, 0.0,...",1
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
4,"[0.0, 0.0, 0.0, 0.0, 28.147108, 0.0, 0.0, 0.0,...",1


# Evaluation Metrics

In [16]:
def compute_accuracy(actual_class,predicted_class):
  number_of_correctly_classified=0
  for i in range(len(actual_class)):
    if actual_class[i] == predicted_class[i]:
      number_of_correctly_classified += 1
  accuracy = number_of_correctly_classified / len(actual_class)
  return accuracy

In [17]:
def compute_FAR(actual_class,predicted_class):
  number_of_forged_instances_classified_as_genuine = 0
  for i in range(len(actual_class)):
    if actual_class[i] == 0 and predicted_class[i] == 1:
      number_of_forged_instances_classified_as_genuine += 1
  far = number_of_forged_instances_classified_as_genuine / len(actual_class)
  return far

In [20]:
def compute_FRR(actual_class,predicted_class):
  number_of_genuine_instances_classified_as_forged = 0
  for i in range(len(actual_class)):
    if actual_class[i] == 1 and predicted_class[i] == 0:
      number_of_genuine_instances_classified_as_forged += 1
  frr = number_of_genuine_instances_classified_as_forged / len(actual_class)
  return frr

# Training and Testing

In [21]:
svclassifier = SVC(kernel='linear')
classifier = svclassifier.fit(list(training_data["signature"]), list(training_data["signature_class"]))
obtained_class = classifier.predict(list(testing_data["signature"]))
accuracy = compute_accuracy(testing_data["signature_class"],obtained_class)
FAR = compute_FAR(testing_data["signature_class"],obtained_class)
FRR = compute_FRR(testing_data["signature_class"],obtained_class)
print(accuracy, FAR, FRR)

0.71875 0.06944444444444445 0.21180555555555555


In [22]:
classifier = LogisticRegression(random_state=0).fit(list(training_data["signature"]),list(training_data["signature_class"])) 
obtained_class = classifier.predict(list(testing_data["signature"]))
accuracy = compute_accuracy(testing_data["signature_class"],obtained_class)
FAR = compute_FAR(testing_data["signature_class"],obtained_class)
FRR = compute_FRR(testing_data["signature_class"],obtained_class)
print(accuracy, FAR, FRR)

0.7291666666666666 0.06770833333333333 0.203125


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
knn = KNeighborsClassifier(n_neighbors=7)
classifier = knn.fit(list(training_data["signature"]),list(training_data["signature_class"]))
accuracy = compute_accuracy(testing_data["signature_class"],obtained_class)
FAR = compute_FAR(testing_data["signature_class"],obtained_class)
FRR = compute_FRR(testing_data["signature_class"],obtained_class)
print(accuracy, FAR, FRR)

0.7291666666666666 0.06770833333333333 0.203125


# Using Pickle

In [24]:
import pickle
pickle.dump(classifier, open('LR_model_pickle.pkl','wb'))

In [25]:
model = pickle.load(open('LR_model_pickle.pkl','rb'))
img = data_original[0]
stacked_img = np.dstack((img, img, img))
input_img = np.expand_dims(stacked_img, axis = 0)
feature_matrix = base_model.predict(input_img)
flattened_matrix = feature_matrix.reshape(-1)
model.predict([flattened_matrix])

1/1 [==============================] - 1s 515ms/step


array([1])

# Using Flask

In [ ]:
# from flask import Flask, request, jsonify

In [ ]:
# !pip install flask-ngrok

In [ ]:
# from flask_ngrok import run_with_ngrok
# app = Flask(__name__)
# run_with_ngrok(app)

In [ ]:
# @app.route('/api',methods=['POST'])
# def predict():
#     data = request.get_json(force=True)
#     prediction = model.predict([[np.array(data['exp'])]])
#     output = prediction[0]
#     return jsonify(output)

In [ ]:
# @app.route('/')
# def home():
#     return "<h1>Hello</h1>"

In [ ]:
# if __name__ == '__main__':
#   app.run()

# Using Request

In [ ]:
# import requests
# url = 'http://localhost:5000/api'
# r = requests.post(url,json={'exp':1.8,})
# print(r.json())